# 0. DataSetting

In [1]:
#import packages & libraries
import pandas as pd
import os
import glob
from sklearn.model_selection import StratifiedKFold
from scipy.ndimage import rotate
import numpy as np
import pydicom
from skimage import measure
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import h5py

In [2]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU 가속 사용 가능")
else:
    device = torch.device("cpu")
    print("GPU 가속 사용 불가능")

# 간단한 연산을 GPU에서 실행
a = torch.tensor([1.0, 2.0, 3.0], device=device)
b = torch.tensor([4.0, 5.0, 6.0], device=device)
c = a + b

print("연산 결과:", c)

GPU 가속 사용 가능
연산 결과: tensor([5., 7., 9.], device='cuda:0')


In [2]:
num_cores = os.cpu_count()
print(f"Number of available cores: {num_cores}")

Number of available cores: 48


In [3]:
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")

NameError: name 'torch' is not defined

## 0-1 Ground Truh Label Setting

In [3]:
# 주 데이터 폴더 경로
main_data_folder = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast"
image_data_folder = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/401~514/401_514_screen"

# 세부 폴더의 이름
sub_folder_name = '401~514'

# 개별 환자 폴더 설정
patient_folder_names = [f for f in os.listdir(image_data_folder) if os.path.isdir(os.path.join(image_data_folder, f))]

# Find the first Excel file in the directory
excel_file_path = glob.glob(f"{main_data_folder}/{sub_folder_name}/*.xlsx")[0]
excel_data = pd.read_excel(excel_file_path)

labels = excel_data['Molecular subtype(1 : luminal A, 2 : luminal B, 3: Her2 positive, 4: Triple negative)'].values

## 0-2 Roi Dataset Setting

In [4]:
# 병변을 나타내는 색상들의 RGB 값
lesion_colors = np.array([
    [255, 0, 0],      # Red
    [255, 255, 0],    # Yellow1
    [255, 102, 0],    # Yellow2
    [0, 102, 255],   # Blue1
    [0, 0, 255],     # Blue2
    [0, 255, 0],    # Green
    [128, 0, 255]   # Purple
])

# DICOM 파일들을 읽고 이미지 데이터를 가져오는 함수
def load_patient_images(patient_folder):
    
    # patient_folder 내의 DICOM 파일들을 가져오기
    dicom_files = [os.path.join(patient_folder, f) for f in os.listdir(patient_folder) if f.endswith('.dcm')]

    # DICOM 파일들을 읽기
    dicoms = [pydicom.dcmread(d) for d in dicom_files]

    # 각 DICOM 파일의 이미지 위치 정보를 기준으로 정렬
    dicoms.sort(key=lambda d: float(d.ImagePositionPatient[2]))

    # 각 DICOM 파일의 pixel_array를 가져오기
    images = np.stack([d.pixel_array for d in dicoms])

    return images

# 병변 영역을 찾는 함수
def find_lesions(images):
    lesion_mask = np.any([np.all(images == color, axis=-1) for color in lesion_colors], axis=0)
    return lesion_mask

# 가장 큰 병변의 Bounding Volume을 찾는 함수
def find_largest_lesion_and_bounding_volume(lesion_mask):
    
    # 3D Connected Component Labeling 수행
    labels = measure.label(lesion_mask)

    # 각 병변의 크기를 계산하고 가장 큰 병변을 찾기
    sizes = np.bincount(labels.ravel())
    max_label = sizes[1:].argmax() + 1  # 0 레이블은 배경이므로 제외

    # 가장 큰 병변에 해당하는 픽셀들의 좌표를 찾기
    lesion_pixels = np.where(labels == max_label)

    # bounding volume의 모서리를 찾기
    min_corner = np.array([np.min(i) for i in lesion_pixels])
    max_corner = np.array([np.max(i) for i in lesion_pixels])

    return min_corner, max_corner

# DICOM 파일들을 읽고 이미지 데이터를 가져오는 함수
def load_dicoms(patient_folder):
    
    # patient_folder 내의 DICOM 파일들을 가져오기
    dicom_files = [os.path.join(patient_folder, f) for f in os.listdir(patient_folder) if f.endswith('.dcm')]

    # DICOM 파일들을 읽기
    dicoms = [pydicom.dcmread(d) for d in dicom_files]

    # 각 DICOM 파일의 이미지 위치 정보를 기준으로 정렬
    dicoms.sort(key=lambda d: float(d.ImagePositionPatient[2]))

    # 각 DICOM 파일의 pixel_array를 가져오기
    images = np.stack([d.pixel_array for d in dicoms])

    return images, dicoms

# images를 3D 복셀로 변환합니다.
def create_voxel(images, dicoms):
    
    # 모든 2D DICOM 이미지를 하나의 3D numpy 배열로 결합
    voxel = images.astype(float)

    # DICOM 파일의 RescaleIntercept와 RescaleSlope를 적용하여 픽셀 값 조정
    voxel *= np.float64(dicoms[0].RescaleSlope)
    voxel += np.float64(dicoms[0].RescaleIntercept)

    return voxel

In [5]:
def process_patient_folder(patient_folder_name):
    try:
        print(f"Processing folder: {patient_folder_name}")
        
        patient_folder_path = os.path.join(image_data_folder, patient_folder_name) # 환자 폴더 경로
        folder_list = os.listdir(patient_folder_path)

        # 환자의 일련번호를 patient_num 변수에 저장
        patient_num = int(patient_folder_name)

        # 'cad'와 'precontrast' 폴더 찾기
        cad_folder_name = next(f for f in folder_list if 'cad' in f.lower())
        precontrast_folder_name = next(f for f in folder_list if 'precontrast' in f.lower())

        # 경로 생성
        cad_folder = os.path.join(patient_folder_path, cad_folder_name)
        precontrast_folder = os.path.join(patient_folder_path, precontrast_folder_name)

        # 각 폴더의 하위 폴더 찾기
        cad_subfolder_name = next(f for f in os.listdir(cad_folder) if os.path.isdir(os.path.join(cad_folder, f)))
        precontrast_subfolder_name = next(f for f in os.listdir(precontrast_folder) if os.path.isdir(os.path.join(precontrast_folder, f)))

        # DICOM 폴더 경로 생성
        dicom_folder = os.path.join(cad_folder, cad_subfolder_name)
        dicom_folder_pre = os.path.join(precontrast_folder, precontrast_subfolder_name)

        # DICOM 이미지들을 로드하기
        images = load_patient_images(dicom_folder)

        # 병변 마스크를 생성하기
        lesion_mask = find_lesions(images)

        # 가장 큰 병변의 Bounding Volume 찾기
        min_corner, max_corner = find_largest_lesion_and_bounding_volume(lesion_mask)

        # DICOM 이미지를 로드하기
        images_pre, dicoms_pre = load_dicoms(dicom_folder_pre)

        # images_pre를 3D 복셀로 변환합니다.
        voxels_pre = create_voxel(images_pre, dicoms_pre)

        # precontrast MRI에서 해당 ROI 잘라내기
        roi_precontrast = voxels_pre[min_corner[0]:max_corner[0], min_corner[1]:max_corner[1], min_corner[2]:max_corner[2]]

        return patient_num, roi_precontrast
    
    except Exception as e:
        print(f"Failed processing folder: {patient_folder_name}")
        raise e
# HDF5 파일로 저장하기 위한 h5py.File 인스턴스 생성
save_folder = '/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/401~514/hdf5_401-514' # 저장될 폴더 경로 설정
hdf5_file_name = f'{sub_folder_name}_data.h5' # 저장될 HDF5 파일 이름 설정

with h5py.File(os.path.join(save_folder, hdf5_file_name), 'w') as hdf5_file:
    with concurrent.futures.ProcessPoolExecutor(max_workers=30) as executor:
        # 각 환자 폴더에 대한 작업을 시작합니다.
        futures_to_folder = {executor.submit(process_patient_folder, patient_folder_name): patient_folder_name for patient_folder_name in patient_folder_names}

        for future in concurrent.futures.as_completed(futures_to_folder):
            patient_folder_name = futures_to_folder[future]
            try:
                patient_num, roi_precontrast = future.result()
                # roi_precontrast를 해당 환자의 번호로 HDF5 파일에 저장
                hdf5_file.create_dataset(str(patient_num), data=roi_precontrast, chunks=True)
            except Exception as e:
                print(f"Failed processing folder: {patient_folder_name} with error: {e}")

Processing folder: 508Processing folder: 445Processing folder: 418
Processing folder: 426Processing folder: 425Processing folder: 410Processing folder: 413
Processing folder: 469





Processing folder: 504Processing folder: 403
Processing folder: 488Processing folder: 503Processing folder: 464Processing folder: 498Processing folder: 402
Processing folder: 411Processing folder: 477Processing folder: 428Processing folder: 451Processing folder: 482
Processing folder: 454

Processing folder: 478Processing folder: 422Processing folder: 456Processing folder: 453Processing folder: 466Processing folder: 442Processing folder: 455Processing folder: 505












Processing folder: 486



Processing folder: 484
Processing folder: 431
Processing folder: 415
Processing folder: 467
Processing folder: 417
Processing folder: 489
Processing folder: 509Processing folder: 419

Processing folder: 450
Processing folder: 457
Processing folder: 420
Processing folder: 424
Processing folder: 446
Processing 

In [13]:
import h5py

# 저장된 HDF5 파일 열기
with h5py.File(os.path.join(save_folder, hdf5_file_name), 'r') as hdf5_file:
   
    # 파일 내에 저장된 모든 데이터셋의 이름 출력
    for key in hdf5_file.keys():
        print(key)
        
        # 각 데이터셋의 shape 출력
        print(hdf5_file[key].shape)

1
(143, 87, 142)
10
(22, 43, 32)
100
(35, 22, 56)
11
(34, 77, 42)
12
(81, 100, 88)
13
(33, 44, 35)
14
(70, 34, 77)
15
(38, 75, 62)
16
(48, 74, 44)
17
(30, 80, 45)
18
(85, 32, 103)
19
(12, 17, 22)
2
(56, 26, 99)
20
(28, 60, 38)
21
(19, 31, 33)
22
(25, 41, 42)
23
(22, 47, 51)
24
(134, 104, 102)
25
(27, 31, 26)
26
(67, 29, 93)
27
(58, 34, 77)
28
(15, 40, 17)
29
(83, 158, 184)
3
(48, 75, 46)
30
(122, 93, 76)
31
(153, 159, 160)
32
(40, 53, 38)
33
(68, 48, 50)
34
(65, 75, 56)
35
(55, 64, 201)
36
(106, 74, 52)
37
(35, 68, 53)
38
(117, 98, 135)
39
(97, 133, 132)
4
(40, 47, 30)
40
(47, 102, 73)
41
(86, 132, 108)
42
(54, 103, 90)
43
(35, 45, 39)
44
(58, 30, 76)
45
(47, 36, 30)
46
(46, 35, 60)
47
(25, 32, 25)
48
(48, 55, 41)
49
(141, 134, 59)
5
(14, 29, 55)
50
(29, 42, 50)
51
(112, 68, 146)
52
(34, 62, 64)
53
(168, 48, 28)
54
(97, 90, 186)
55
(84, 50, 106)
56
(13, 21, 21)
57
(87, 46, 129)
58
(67, 64, 54)
59
(43, 42, 108)
6
(80, 39, 125)
60
(58, 116, 191)
61
(42, 96, 111)
62
(71, 81, 89)
63
(32, 2

## 512명(227번, 437번 제외) 중 가장 큰 볼륨을 갖는 파일 찾기

In [3]:
import os
import h5py
import numpy as np

base_paths = [
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/001~100/hdf5_001-100",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/101~200/hdf5_101-200",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/201~300/hdf5_201-300",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/301~400/hdf5_301-400",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/401~514/hdf5_401-514"
]

max_shape = (0, 0, 0)

for base_path in base_paths:
    for filename in os.listdir(base_path):
        if filename.endswith(".h5"):
            with h5py.File(os.path.join(base_path, filename), 'r') as f:
                # 모든 데이터셋에 대해서 shape을 확인
                for key in f.keys():
                    current_shape = f[key].shape
                    max_shape = tuple(np.maximum(max_shape, current_shape))

print("Maximum shape found:", max_shape)

Maximum shape found: (168, 286, 417)


## padding

In [8]:
import os
import h5py
import numpy as np

base_paths = [
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/001~100/hdf5_001-100",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/101~200/hdf5_101-200",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/201~300/hdf5_201-300",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/301~400/hdf5_301-400",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/401~514/hdf5_401-514"
]

def pad_data(data, target_shape):
    pad_values = [(int((target_shape[i] - data.shape[i]) / 2), 
                   int((target_shape[i] - data.shape[i] + 1) / 2)) for i in range(3)]
    return np.pad(data, pad_values, mode='constant')

for base_path in base_paths:
    files = [f for f in os.listdir(base_path) if f.endswith('.h5')]
    
    # 새로운 저장 경로 설정
    store_path = os.path.join(os.path.dirname(base_path), 'padded_' + os.path.basename(base_path))
    os.makedirs(store_path, exist_ok=True)

    for file in files:
        with h5py.File(os.path.join(base_path, file), 'r') as hf:
            # 여러 데이터셋에 대해서 패딩 적용
            for key in hf.keys():
                data = hf[key][:]
                padded_data = pad_data(data, (168, 286, 417))
                
                # 저장될 파일 경로
                save_path = os.path.join(store_path, 'padded_' + file)
                with h5py.File(save_path, 'a') as sf: 
                    if key in sf:
                        del sf[key]
                    sf.create_dataset(key, data=padded_data)

print("All data has been padded and stored.")

All data has been padded and stored.


## padded data check

In [10]:
import os
import h5py

base_paths = [
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/001~100/padded_hdf5_001-100",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/101~200/padded_hdf5_101-200",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/201~300/padded_hdf5_201-300",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/301~400/padded_hdf5_301-400",
    "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/401~514/padded_hdf5_401-514"
]

# 각 base_path에 대하여
for base_path in base_paths:
    hdf5_files = [f for f in os.listdir(base_path) if f.endswith('.h5')]

    for hdf5_file_name in hdf5_files:
        # 저장된 HDF5 파일 열기
        with h5py.File(os.path.join(base_path, hdf5_file_name), 'r') as hdf5_file:

            # 파일 내에 저장된 모든 데이터셋의 이름 출력
            for key in hdf5_file.keys():
                print(f"File: {hdf5_file_name}, Dataset Name: {key}")
                
                # 각 데이터셋의 shape 출력
                print(f"Shape: {hdf5_file[key].shape}")

File: padded_001~100_data.h5, Dataset Name: 1
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 10
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 100
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 11
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 12
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 13
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 14
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 15
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 16
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 17
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 18
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 19
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 2
Shape: (168, 286, 417)
File: padded_001~100_data.h5, Dataset Name: 20
Shape: (168, 286, 417)
File: padded_001~100_